# 01 - Load data into an Azure SQL heap, non-partitioned, non-indexed, table

In Azure SQL terminology an Heap is a table with no clustered index. In this samples we'll load data into a table that as no index (clustered or non-clustered) as is not partitioned. This is the simplest scenario possibile and allows parallel load of data.

Sample is using both the new sql-spark-connector (https://github.com/microsoft/sql-spark-connector), and the previous one (https://github.com/Azure/azure-sqldb-spark). To install the _new connector_ manually import the .jar file (available in GitHub repo's releases) into the cluster. To install the previous one, just import the library right from Databricks portal using the "com.microsoft.azure:azure-sqldb-spark:1.0.2" coordinates.

Define variables used thoughout the script. Azure Key Value has been used to securely store sensitive data. More info here: [Create an Azure Key Vault-backed secret scope](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes#--create-an-azure-key-vault-backed-secret-scope)

In [3]:
val scope = "key-vault-secrets"

val storageAccount = "dmstore2";
val storageKey = dbutils.secrets.get(scope, "dmstore2-2");

val server = dbutils.secrets.get(scope, "srv001").concat(".database.windows.net");
val database = dbutils.secrets.get(scope, "db001");
val user = dbutils.secrets.get(scope, "dbuser001");
val password = dbutils.secrets.get(scope, "dbpwd001");
val table = "dbo.LINEITEM_LOADTEST"


scope: String = key-vault-secrets
storageAccount: String = dmstore2
storageKey: String = [REDACTED]
server: String = [REDACTED].database.windows.net
database: String = [REDACTED]
user: String = [REDACTED]
password: String = [REDACTED]
table: String = dbo.LINEITEM_LOADTEST

Configure Spark to access Azure Blob Store

In [5]:
spark.conf.set(s"fs.azure.account.key.$storageAccount.blob.core.windows.net", storageKey);

Load the Parquet file generated in `00-create-parquet-file` notebook that contains LINEITEM data partitioned by Year and Month

In [7]:
val li = spark.read.parquet("wasbs://tpch@dmstore2.blob.core.windows.net/10GB/parquet/lineitem")

Loaded data is split in 20 dataframe partitions

In [9]:
li.rdd.getNumPartitions

res2: Int = 20

Show schema of loaded data

In [11]:
li.printSchema

root
-- L_ORDERKEY: integer (nullable = true)
-- L_PARTKEY: integer (nullable = true)
-- L_SUPPKEY: integer (nullable = true)
-- L_LINENUMBER: integer (nullable = true)
-- L_QUANTITY: decimal(15,2) (nullable = true)
-- L_EXTENDEDPRICE: decimal(15,2) (nullable = true)
-- L_DISCOUNT: decimal(15,2) (nullable = true)
-- L_TAX: decimal(15,2) (nullable = true)
-- L_RETURNFLAG: string (nullable = true)
-- L_LINESTATUS: string (nullable = true)
-- L_SHIPDATE: date (nullable = true)
-- L_COMMITDATE: date (nullable = true)
-- L_RECEIPTDATE: date (nullable = true)
-- L_SHIPINSTRUCT: string (nullable = true)
-- L_SHIPMODE: string (nullable = true)
-- L_COMMENT: string (nullable = true)
-- L_PARTITION_KEY: integer (nullable = true)

All columns are shown as nullable, even if they were originally set to NOT NULL, so we will need to keep this in mind later.

Make sure you create on your Azure SQL the following LINEITEM table:
```sql
create table [dbo].[LINEITEM_LOADTEST]
(
	[L_ORDERKEY] [int] not null,
	[L_PARTKEY] [int] not null,
	[L_SUPPKEY] [int] not null,
	[L_LINENUMBER] [int] not null,
	[L_QUANTITY] [decimal](15, 2) not null,
	[L_EXTENDEDPRICE] [decimal](15, 2) not null,
	[L_DISCOUNT] [decimal](15, 2) not null,
	[L_TAX] [decimal](15, 2) not null,
	[L_RETURNFLAG] [char](1) not null,
	[L_LINESTATUS] [char](1) not null,
	[L_SHIPDATE] [date] not null,
	[L_COMMITDATE] [date] not null,
	[L_RECEIPTDATE] [date] not null,
	[L_SHIPINSTRUCT] [char](25) not null,
	[L_SHIPMODE] [char](10) not null,
	[L_COMMENT] [varchar](44) not null,
	[L_PARTITION_KEY] [int] not null
) 
```

## Using the new connector

Schema needs to be defined explicitly as new connector is very sensitive to nullability, as per the following issue [Nullable column mismatch between Spark DataFrame & SQL Table Error](
https://github.com/microsoft/sql-spark-connector/issues/5), so we need to explicity create the schema and apply it to the loaded data

In [16]:
import org.apache.spark.sql.types._

val schema = StructType(
    StructField("L_ORDERKEY", IntegerType, false) ::
    StructField("L_PARTKEY", IntegerType, false) ::
    StructField("L_SUPPKEY", IntegerType, false) ::  
    StructField("L_LINENUMBER", IntegerType, false) ::
    StructField("L_QUANTITY", DecimalType(15,2), false) ::
    StructField("L_EXTENDEDPRICE", DecimalType(15,2), false) ::
    StructField("L_DISCOUNT", DecimalType(15,2), false) ::
    StructField("L_TAX", DecimalType(15,2), false) ::
    StructField("L_RETURNFLAG", StringType, false) ::
    StructField("L_LINESTATUS", StringType, false) ::
    StructField("L_SHIPDATE", DateType, false) ::
    StructField("L_COMMITDATE", DateType, false) ::
    StructField("L_RECEIPTDATE", DateType, false) ::
    StructField("L_SHIPINSTRUCT", StringType, false) ::  
    StructField("L_SHIPMODE", StringType, false) ::  
    StructField("L_COMMENT", StringType, false) ::  
    StructField("L_PARTITION_KEY", IntegerType, false) ::  
    Nil)
    
  val li2 = spark.createDataFrame(li.rdd, schema)

import org.apache.spark.sql.types._
schema: org.apache.spark.sql.types.StructType = StructType(StructField(L_ORDERKEY,IntegerType,false), StructField(L_PARTKEY,IntegerType,false), StructField(L_SUPPKEY,IntegerType,false), StructField(L_LINENUMBER,IntegerType,false), StructField(L_QUANTITY,DecimalType(15,2),false), StructField(L_EXTENDEDPRICE,DecimalType(15,2),false), StructField(L_DISCOUNT,DecimalType(15,2),false), StructField(L_TAX,DecimalType(15,2),false), StructField(L_RETURNFLAG,StringType,false), StructField(L_LINESTATUS,StringType,false), StructField(L_SHIPDATE,DateType,false), StructField(L_COMMITDATE,DateType,false), StructField(L_RECEIPTDATE,DateType,false), StructField(L_SHIPINSTRUCT,StringType,false), StructField(L_SHIPMODE,StringType,false), StructField(L_COMMENT,StringType,false), StructField(L_PARTITION_KEY,IntegerType,false))
li2: org.apache.spark.sql.DataFrame = [L_ORDERKEY: int, L_PARTKEY: int ... 15 more fields]

In [17]:
val url = s"jdbc:sqlserver://$server;databaseName=$database;"

li2.write 
  .format("com.microsoft.sqlserver.jdbc.spark") 
  .mode("overwrite")   
  .option("truncate", "true") 
  .option("url", url) 
  .option("dbtable", table) 
  .option("user", user) 
  .option("password", password) 
  .option("reliabilityLevel", "BEST_EFFORT") 
  .option("tableLock", "true") 
  .option("batchsize", "100000") 
  .save()

url: String = jdbc:sqlserver://[REDACTED].database.windows.net;databaseName=[REDACTED];

# Using the old connector:

This connector is more permissive about schema so we can just use the schema coming from Parquet file

In [20]:
import com.microsoft.azure.sqldb.spark.bulkcopy.BulkCopyMetadata
import com.microsoft.azure.sqldb.spark.config.Config
import com.microsoft.azure.sqldb.spark.connect._

val config = Config(Map(
  "url" -> server,
  "databaseName" -> database,
  "dbTable" -> table,
  "user" -> user,
  "password" -> password,
  "bulkCopyBatchSize" -> "100000",
  "bulkCopyTableLock" -> "true",  
  "bulkCopyTimeout" -> "600" //seconds  
))

li.bulkCopyToSqlDB(config)

import com.microsoft.azure.sqldb.spark.bulkcopy.BulkCopyMetadata
import com.microsoft.azure.sqldb.spark.config.Config
import com.microsoft.azure.sqldb.spark.connect._
config: com.microsoft.azure.sqldb.spark.config.Config = com.microsoft.azure.sqldb.spark.config.ConfigBuilder$$anon$1@b4be6705